In [1]:
from enum import Enum

class Simple(Enum):
    ONE = '1'
    TWO = '2'

class StrEnum(str, Enum):
    pass

class Shape(StrEnum):
    OFFSUIT = 'o'
    SUITED = 's'
    PAIR = ''
    ELSE = 1

In [23]:
%timeit 'o' in Shape.__members__.values()

100000 loops, best of 3: 4.65 µs per loop


In [11]:
Simple.ONE == '1'

False

In [12]:
Simple._member_type_

object

In [ ]:
StrEnum

In [16]:
Shape('s')

<Shape.SUITED: 's'>

In [22]:
Shape('s') == Shape.SUITED == 's'

True

In [23]:
Shape._member_type_

str

In [24]:
Shape._member_map_

OrderedDict([('OFFSUIT', <Shape.OFFSUIT: 'o'>), ('SUITED', <Shape.SUITED: 's'>), ('PAIR', <Shape.PAIR: ''>), ('ELSE', <Shape.ELSE: '1'>)])

In [20]:
Shape(1)

ValueError: 1 is not a valid Shape

In [4]:
Shape('1')

<Shape.ELSE: '1'>

In [5]:
list(Shape)

[<Shape.OFFSUIT: 'o'>,
 <Shape.SUITED: 's'>,
 <Shape.PAIR: ''>,
 <Shape.ELSE: '1'>]

In [6]:
isinstance(Shape.ELSE, str)

True

In [7]:
Shape.OFFSUIT

<Shape.OFFSUIT: 'o'>

In [8]:
Shape.OFFSUIT.value

'o'

In [9]:
type(Shape.ELSE.value)

str

In [10]:
from poker.hand import Shape as PokerShape

In [11]:
PokerShape.OFFSUIT.value

'o'

In [12]:
PokerShape.OFFSUIT

Shape('o')

In [13]:
isinstance(PokerShape.OFFSUIT, str)

False

In [14]:
Shape.ELSE == '1'

True

In [17]:
class SimpleInt(int, Enum):
    one = 1
    two = '2'

In [18]:
SimpleInt(2)

<SimpleInt.two: 2>

In [19]:
SimpleInt.two == '2'

False

In [20]:
SimpleInt.two == 2

True

In [6]:
from enum34_custom import StrEnum

In [1]:
from enum34_custom import MultiValueEnum2

(<class 'str'>, <enum 'Enum'>) Ellipsis


In [4]:
class MyTupleEnum(MultiValueEnum2):
    a = 1
    b = 1, 2, 3

In [5]:
MyTupleEnum.a

<MyTupleEnum.a: (1,)>

In [6]:
MyTupleEnum.b

<MyTupleEnum.b: (1, 2, 3)>

In [8]:
MyTupleEnum((1, 2, 3))

<MyTupleEnum.b: (1, 2, 3)>

In [9]:
from enum import Enum

class TupleEnum(tuple, Enum):
    a = (1, 4)
    b = (2, 3)

In [10]:
TupleEnum.a

<TupleEnum.a: (1, 4)>

In [11]:
TupleEnum.a == (1, 4)

True

In [12]:
TupleEnum(1)

ValueError: 1 is not a valid TupleEnum

In [27]:
class StrEnum(str, Enum):
    'accepts only string values'

class Name(StrEnum):
    BDFL = 'Guido van Rossum'
    FLUFL = 'Barry Warsaw'
    ABC = 1

Name.ABC

<Name.ABC: '1'>

In [30]:
'1' < '2' < '3' > '4'

False

In [31]:
'1' < 'A'

True

# Poker vs custom

In [1]:
from enum import EnumMeta, Enum
from types import DynamicClassAttribute
from functools import total_ordering


class _MultiMeta(EnumMeta):
    def __init__(enum_class, cls, bases, classdict):
        # make sure we only have tuple values, not single values
        for member in enum_class.__members__.values():
            if not isinstance(member._value_, tuple):
                raise ValueError('{!r}, should be tuple'.format(member._value_))

    def __call__(cls, suit):
        for member in cls:
            if suit in member._value_:
                return member
        return super().__call__(suit)

    def make_random(cls):
        return random.choice(list(cls))


@total_ordering
class _MultiValueEnum(Enum, metaclass=_MultiMeta):
    # pass
    # From Python manual:
    # If a class that overrides __eq__() needs to retain the implementation of __hash__()
    # from a parent class, the interpreter must be told this explicitly
    def __hash__(self):
        return super().__hash__()

    def __eq__(self, other):
        if self.__class__ is other.__class__:
            return self._value_ == other._value_
        return NotImplemented

    def __lt__(self, other):
        if self.__class__ is other.__class__:
            names = self.__class__._member_names_
            return names.index(self.name) < names.index(other.name)
        return NotImplemented

    def __str__(self):
        return str(self.value)

    # def __repr__(self):
        # apostrophe = "'" if isinstance(self.value, str) else ''
        # return "{0}({1}{2}{1})".format(self.__class__.__qualname__, apostrophe, self)

    @DynamicClassAttribute
    def value(self):
        """The value of the Enum member."""
        return self._value_[0]

In [2]:
class Suit(_MultiValueEnum):
    CLUBS =    '♣', 'c', 'C', 'clubs'
    DIAMONDS = '♦', 'd', 'D', 'diamonds'
    HEARTS =   '♥', 'h', 'H', 'hearts'
    SPADES =   '♠', 's', 'S', 'spades'
    C = '♣', 'c', 'C', 'clubs'
    D = '♦', 'd', 'D', 'diamonds'
    H = HEARTS
    S = SPADES

In [3]:
Suit.C

<Suit.C: ('♣', 'c', 'C', 'clubs')>

## Custom

In [4]:
from enum import Enum, EnumMeta
from collections import Iterable

class _MultiValueMeta2(EnumMeta):
    def __init__(self, clsname, bases, classdict):
        # make sure we only have tuple values, not single values
        for member in self.__members__.values():
            val = member.value
            if not isinstance(val, Iterable) or isinstance(val, str):
                raise TypeError('{} = {!r}, should be iterable, not {}!'
                    .format(member.name, val, type(val))
                )
            # set is faster to lookup
            member._lookup_set_ = set(val)

    def __call__(cls, value):
        """Return the appropriate instance with any of the values listed."""
        for member in cls:
            if value in member._lookup_set_:
                return member
        else:
            raise ValueError("%s is not a valid %s" % (value, cls.__name__))


class MultiValueEnum2(Enum, metaclass=_MultiValueMeta2):
    """Enum subclass where a member can be any iterable (except str).
    You can reference a member by any of its element in the associated iterable.
    """


In [5]:
class Suit2(MultiValueEnum2):
    CLUBS =    '♣', 'c', 'C', 'clubs'
    DIAMONDS = '♦', 'd', 'D', 'diamonds'
    HEARTS =   '♥', 'h', 'H', 'hearts'
    SPADES =   '♠', 's', 'S', 'spades'
    C = '♣', 'c', 'C', 'clubs'
    D = '♦', 'd', 'D', 'diamonds'
    H = HEARTS
    S = SPADES

In [6]:
Suit2.C

<Suit2.CLUBS: ('♣', 'c', 'C', 'clubs')>

# Bugreport

In [3]:
from enum import EnumMeta, Enum
from types import DynamicClassAttribute


class _MultiMeta(EnumMeta):
    def __init__(enum_class, cls, bases, classdict):
        # make sure we only have tuple values, not single values
        for member in enum_class.__members__.values():
            if not isinstance(member._value_, tuple):
                raise ValueError('{!r}, should be tuple'.format(member._value_))

    def __call__(cls, suit):
        for member in cls:
            if suit in member._value_:
                return member
        return super().__call__(suit)


class MultiValueEnum(Enum, metaclass=_MultiMeta):
    @DynamicClassAttribute
    def value(self):
        """The value of the Enum member."""
        return self._value_[0]

class IncorrectAliasBehavior(MultiValueEnum):
    first = 1, 2, 3
    second = 4, 5, 6
    alias_to_first = 1, 2, 3

In [4]:
IncorrectAliasBehavior.alias_to_first

<IncorrectAliasBehavior.alias_to_first: (1, 2, 3)>

# 

In [1]:
>>> from enum import Enum
>>> class Color(Enum):
...     red = 1
...     green = 2
...     blue = 3

In [2]:
Color(Color.red)

<Color.red: 1>

In [3]:
Color['red']

<Color.red: 1>

In [4]:
Color['green']

<Color.green: 2>

In [6]:
Color(Color(Color.red))

<Color.red: 1>

# Pickable

In [1]:
import pickle
from enum34_custom import OrderableMixin, MultiValueEnum


class Color(OrderableMixin, MultiValueEnum):
    red = 1, 2
    green = 3, 4
    blue = 1, 2 
    yellow = red

pickle.dumps(Color.red)

b'\x80\x03c__main__\nColor\nq\x00K\x01K\x02\x86q\x01\x85q\x02Rq\x03.'

In [2]:
red = pickle.loads(pickle.dumps(Color.red))

In [3]:
red

<Color.red: (1, 2)>

In [4]:
red < Color.green

True

In [5]:
red >Color.green

False

In [6]:
list(Color)

[<Color.red: (1, 2)>, <Color.green: (3, 4)>]

# Compare

In [3]:
from enum34_custom import MultiValueEnum, OrderableMixin

class Color(OrderableMixin, MultiValueEnum):
    red = 'r', 'red', '#F00'
    green = 'g', 'green', '#0F0'
    

Color.red == 'red'
Color.red == 'r'
Color.red == '#F00'

False

# Much simpler MultivalueEnum

In [10]:
from enum import Enum
from types import DynamicClassAttribute


class MultiValueEnum(Enum):
    def __new__(cls, *values):
        print('NEW:', cls, values)
        obj = object.__new__(cls)
        obj._value_ = values
        obj._all_values = values
        for alias in values:
            cls._value2member_map_[alias] = obj
        return obj
    
    def __repr__(self):
        return "<%s.%s: %s>" % (
             self.__class__.__name__,
             self._name_,
             ', '.join(["'%s'" % v for v in self._all_values])
             )
    
    @DynamicClassAttribute
    def value(self):
        return self._value_[0]

In [11]:
class Suit(MultiValueEnum):
    CLUBS =    '♣', 'c', 'C', 'clubs', 'club'
    DIAMONDS = '♦', 'd', 'D', 'diamonds', 'diamond'
    HEARTS =   '♥', 'h', 'H', 'hearts', 'heart'
    SPADES =   '♠', 's', 'S', 'spades', 'spade'
    S = SPADES

NEW: <enum 'Suit'> ('♣', 'c', 'C', 'clubs', 'club')
NEW: <enum 'Suit'> ('♦', 'd', 'D', 'diamonds', 'diamond')
NEW: <enum 'Suit'> ('♥', 'h', 'H', 'hearts', 'heart')
NEW: <enum 'Suit'> ('♠', 's', 'S', 'spades', 'spade')
NEW: <enum 'Suit'> ('♠', 's', 'S', 'spades', 'spade')


In [12]:
Suit('c')

<Suit.CLUBS: '♣', 'c', 'C', 'clubs', 'club'>

In [13]:
Suit._value2member_map_

{'♦': <Suit.DIAMONDS: '♦', 'd', 'D', 'diamonds', 'diamond'>,
 ('♠',
  's',
  'S',
  'spades',
  'spade'): <Suit.S: '♠', 's', 'S', 'spades', 'spade'>,
 'spade': <Suit.S: '♠', 's', 'S', 'spades', 'spade'>,
 ('♣',
  'c',
  'C',
  'clubs',
  'club'): <Suit.CLUBS: '♣', 'c', 'C', 'clubs', 'club'>,
 '♣': <Suit.CLUBS: '♣', 'c', 'C', 'clubs', 'club'>,
 '♥': <Suit.HEARTS: '♥', 'h', 'H', 'hearts', 'heart'>,
 '♠': <Suit.S: '♠', 's', 'S', 'spades', 'spade'>,
 ('♥',
  'h',
  'H',
  'hearts',
  'heart'): <Suit.HEARTS: '♥', 'h', 'H', 'hearts', 'heart'>,
 'd': <Suit.DIAMONDS: '♦', 'd', 'D', 'diamonds', 'diamond'>,
 'diamonds': <Suit.DIAMONDS: '♦', 'd', 'D', 'diamonds', 'diamond'>,
 'H': <Suit.HEARTS: '♥', 'h', 'H', 'hearts', 'heart'>,
 'diamond': <Suit.DIAMONDS: '♦', 'd', 'D', 'diamonds', 'diamond'>,
 'spades': <Suit.S: '♠', 's', 'S', 'spades', 'spade'>,
 'h': <Suit.HEARTS: '♥', 'h', 'H', 'hearts', 'heart'>,
 'club': <Suit.CLUBS: '♣', 'c', 'C', 'clubs', 'club'>,
 'hearts': <Suit.HEARTS: '♥', 'h', 'H', 

In [14]:
Suit.__members__

mappingproxy(OrderedDict([('CLUBS', <Suit.CLUBS: '♣', 'c', 'C', 'clubs', 'club'>), ('DIAMONDS', <Suit.DIAMONDS: '♦', 'd', 'D', 'diamonds', 'diamond'>), ('HEARTS', <Suit.HEARTS: '♥', 'h', 'H', 'hearts', 'heart'>), ('SPADES', <Suit.SPADES: '♠', 's', 'S', 'spades', 'spade'>), ('S', <Suit.S: '♠', 's', 'S', 'spades', 'spade'>)]))

In [15]:
Suit._member_names_

['CLUBS', 'DIAMONDS', 'HEARTS', 'SPADES', 'S']

In [16]:
Suit._member_map_

OrderedDict([('CLUBS', <Suit.CLUBS: '♣', 'c', 'C', 'clubs', 'club'>), ('DIAMONDS', <Suit.DIAMONDS: '♦', 'd', 'D', 'diamonds', 'diamond'>), ('HEARTS', <Suit.HEARTS: '♥', 'h', 'H', 'hearts', 'heart'>), ('SPADES', <Suit.SPADES: '♠', 's', 'S', 'spades', 'spade'>), ('S', <Suit.S: '♠', 's', 'S', 'spades', 'spade'>)])

In [17]:
Suit('s')

<Suit.S: '♠', 's', 'S', 'spades', 'spade'>

In [18]:
class MyBadMultiValueEnum(MultiValueEnum):
    one = 1, 'one'
    two = 2

NEW: <enum 'MyBadMultiValueEnum'> (1, 'one')
NEW: <enum 'MyBadMultiValueEnum'> (2,)


In [19]:
MyBadMultiValueEnum.one.value

1

In [1]:
from enum34_custom import MultiValueEnum

class Col(MultiValueEnum):
    a = 1, 2
    b = 2
    c = 3, 4

TypeError: __new__() takes 2 positional arguments but 3 were given

In [2]:
class StrValues(MultiValueEnum):
    one = ('One'
          'one')
    two = ('two',
          'Two')

VALUES: Oneone


TypeError: 'Oneone', should be iterable, not <class 'str'>!

# Simple enum

In [1]:
from enum import Enum

class ValNum(Enum):
    one
    two
    three

NameError: name 'one' is not defined

In [2]:
from collections import namedtuple

class Valami(namedtuple('Enum', 'one, two')):
    one
    two

NameError: name 'one' is not defined

In [6]:
class SimpleEnumMeta(type):
    def __new__(metacls, clsname, bases, classdict):
        try:
            super().__new__(clsname, bases, classdict)
        except NameError:
            pass

class Sinum(metaclass=SimpleEnumMeta):
    one
    two

NameError: name 'one' is not defined

In [1]:
from simplenum import Enum
>>> class Error(Enum, implicit=False, values=from_one):
...     with implicit:
...         one
...     another_one = 1

ImportError: No module named 'simplenum'

In [5]:
from collections import OrderedDict

def names():
    '''Provide a value for each enum which is the name itself.'''
    def value(name):
        return name
    return value


class ClassDict(OrderedDict):
    '''
    This is the dictionary used while creating Enum instances.  It provides
    default values when `implicit` is true.  This can either be enabled by
    default, or within a `with` context.
    '''

    def __init__(self, implicit=False, values=names):
        '''Setting `implicit` will provide default values from `values`.'''
        super().__init__()
        self.implicit = implicit
        self.always_implicit = implicit
        self.values = values()

    def __enter__(self):
        '''Enable implicit values within a `with` context.'''
        self.implicit = True

    def __exit__(self, exc_type, exc_val, exc_tb):
        '''Disable implicit values on leaving a `with` context.'''
        self.implicit = False
        if exc_val: raise ExplicitError(ERR_MSG) from exc_val

    def __getitem__(self, name):
        '''Provide an item from the dictionary.  Values are created if
        `implicit` is currently true.'''
        if name not in self:
            if self.implicit and Enum and not dunder(name):
                super().__setitem__(name, self.values(name))
                if self.always_implicit: return Explode()
            elif name == 'implicit':
                return self
        return super().__getitem__(name)

    def __setitem__(self, name, value):
        '''Set a value in the dictionary.  Setting is disabled for user
        values (not dunders) if `implicit` is true.  This helps avoid
        confusion from expressions involving shadowed global names.'''
        if self.implicit and Enum and not dunder(name):
            raise ExplicitError('Cannot use explicit value for %s' % name)
        return super().__setitem__(name, value)

    def split(self):
        '''Separate the enums from the special values (dunders and
        descriptors; we assume the latter are methods).'''
        enums, others = OrderedDict(), dict()
        for name in self:
            value = self[name]
            if dunder(name) or hasattr(value, '__get__'):
                others[name] = value
            else:
                enums[name] = value
        return enums, others


class SimpleEnumMeta(type):
    @classmethod
    def __prepare__(metacls, name, bases, values=names):
        '''Provide the class dictionary (which provides implicit values).'''
        print('RUN BEFORE')
        d = ClassDict(values=values)
        print('RUN AFTER:', vars(d))
        return d

    def __init__(metacls, cls, bases, dict, **kargs):
        '''Called during class construction.  Drop kargs.'''
        super().__init__(cls, bases, dict)

    def __new__(metacls, name, bases, prepared, allow_aliases=False, **kargs):
        '''Create the class and then the named tuples, saving the latter in
        the former.'''
        enums, others = prepared.split()
        cls = super().__new__(metacls, name, bases, others)
        cls._enums_by_name, cls._enums_by_value = {}, OrderedDict()
        for name in enums:
            value = enums[name]
            enum = cls.__new__(cls, name, value)
            # handle aliases
            if value in cls._enums_by_value:
                if allow_aliases:
                    cls._enums_by_name[name] = cls._enums_by_value[value]
                else:
                    raise ValueError('Duplicate value (%r) for %s and %s' %
                                     (value, cls._enums_by_value[value].name, name))
            else:
                cls._enums_by_value[value] = enum
                cls._enums_by_name[name] = enum
        # build the delegate from values as that does not include aliases
        cls._enums = MappingProxyType(
                        OrderedDict((enum.name, enum.value)
                                    for enum in cls._enums_by_value.values()))
        return cls
    
    def __getitem__(cls, name): return cls._enums_by_name[name].value
    
    def __getattr__(cls, name):
        '''Provide access to named tuples.'''
        try: return cls._enums_by_name[name]
        except KeyError: raise AttributeError(name)
    def __contains__(cls, name): return name in cls._enums_by_name # aliases

class Sinum(metaclass=SimpleEnumMeta):
    one
    two

RUN BEFORE
RUN AFTER: {'_OrderedDict__hardroot': <collections._Link object at 0x109a87870>, '_OrderedDict__map': {}, 'implicit': False, 'always_implicit': False, 'values': <function names.<locals>.value at 0x109a981e0>, '_OrderedDict__root': <weakproxy at 0x108872688 to _Link at 0x109a87870>}


NameError: name 'one' is not defined

#  Testing class

In [68]:
from enum import Enum
from enum34_custom import MultiValueEnum

class _TournamentAction(Enum):
    REG = 'Tournament Registration'
    UNREG = 'Tournament Unregistration'
    REBUY = 'Tournament Rebuy'
    ADDON = 'Tournament Addon'
    WIN = 'Tournament Won'


class _TransferAction(MultiValueEnum):
    TRANS_IN = 'Transfer Received (Admin)', 'Real Money Transfer Received'
    TRANS_OUT = 'Transfer Sent', 'Real Money Transfer Sent'
    CASHOUT = 'Cashout',
    CASHOUT_CANCEL = 'Cashout Canceled',
    DEPOSIT = 'Deposit (Player)',
    # 'Buy Chips Bonus': ,
    # 'Inter Account Transfer':
    # 'Tournament Ticket Issued':
    # 'Full Tilt Poker Transfer':


class _CreditAction(Enum):
    VIP_SET = 'VIP Status Set'
    FPP_EARNED = 'FPP earned (Tournament)'
    # FPP Store = 'VIP Bonus Purchased'
    # FPP Store = 'Item Purchase (Multiple)'
    # BOUNTY = 'Reward: Knockout Bounty'
    # 'Tournament Ticket Issued':
    # 'Buy In Refunded (Golden SnG)'

class ActionType(MultiValueEnum):
    TOUR = (ta.value for ta in _TournamentAction)
    TRANSFER = (val for tup in _TransferAction for val in tup.value)
    CREDIT = (ca.value for ca in _CreditAction)

In [67]:
[one for ta in _TransferAction for one in ta.value]

['Transfer Received (Admin)',
 'Real Money Transfer Received',
 'Transfer Sent',
 'Real Money Transfer Sent',
 'Cashout',
 'Cashout Canceled',
 'Deposit (Player)']

In [69]:
list(t.value for t in _TournamentAction)

['Tournament Registration',
 'Tournament Unregistration',
 'Tournament Rebuy',
 'Tournament Addon',
 'Tournament Won']

In [73]:
ActionType('Tournament Won') is ActionType.TOUR

True

In [71]:
ActionType('Cashout')

<ActionType.TRANSFER: <generator object <genexpr> at 0x1055b5678>>

In [72]:
ActionType.TOUR

<ActionType.TOUR: <generator object <genexpr> at 0x1055b55a0>>

In [57]:
list(ActionType)

[<ActionType.TOUR: <generator object <genexpr> at 0x105585120>>,
 <ActionType.TRANSFER: <generator object <genexpr> at 0x1055851b0>>,
 <ActionType.CREDIT: <generator object <genexpr> at 0x105585240>>]

In [58]:
ActionType._value2member_map_

{('Transfer Sent',
  'Real Money Transfer Sent'): <ActionType.TRANSFER: <generator object <genexpr> at 0x1055851b0>>,
 <generator object <genexpr> at 0x105585240>: <ActionType.CREDIT: <generator object <genexpr> at 0x105585240>>,
 'Tournament Registration': <ActionType.TOUR: <generator object <genexpr> at 0x105585120>>,
 ('Cashout Canceled',): <ActionType.TRANSFER: <generator object <genexpr> at 0x1055851b0>>,
 ('Cashout',): <ActionType.TRANSFER: <generator object <genexpr> at 0x1055851b0>>,
 'Tournament Addon': <ActionType.TOUR: <generator object <genexpr> at 0x105585120>>,
 ('Deposit (Player)',): <ActionType.TRANSFER: <generator object <genexpr> at 0x1055851b0>>,
 <generator object <genexpr> at 0x105585120>: <ActionType.TOUR: <generator object <genexpr> at 0x105585120>>,
 'FPP earned (Tournament)': <ActionType.CREDIT: <generator object <genexpr> at 0x105585240>>,
 'Tournament Rebuy': <ActionType.TOUR: <generator object <genexpr> at 0x105585120>>,
 ('Transfer Received (Admin)',
  'Rea

In [30]:
from enum34_custom import __version__ as enumver
enumver

'0.6.5'

In [78]:
from collections import namedtuple
_Tournament = namedtuple('_Tournament',
    'date '
    'action '
    'players '
    'game '
    'currency '

    'amount '
    'vpp '
    'fpp '
    'tmoney '
    'wmoney '
    'balance '
    'monthly_vpp '
    'yearly_vpp '
    'total_fpp '
    'total_tmoney '
    'total_wmoney '

    'ident '
    'name '
    'speed '
    'format '
    'limit '
    'type '
    'buyin '
    'rake ',
verbose=True
)


from builtins import property as _property, tuple as _tuple
from operator import itemgetter as _itemgetter
from collections import OrderedDict

class _Tournament(tuple):
    '_Tournament(date, action, players, game, currency, amount, vpp, fpp, tmoney, wmoney, balance, monthly_vpp, yearly_vpp, total_fpp, total_tmoney, total_wmoney, ident, name, speed, format, limit, type, buyin, rake)'

    __slots__ = ()

    _fields = ('date', 'action', 'players', 'game', 'currency', 'amount', 'vpp', 'fpp', 'tmoney', 'wmoney', 'balance', 'monthly_vpp', 'yearly_vpp', 'total_fpp', 'total_tmoney', 'total_wmoney', 'ident', 'name', 'speed', 'format', 'limit', 'type', 'buyin', 'rake')

    def __new__(_cls, date, action, players, game, currency, amount, vpp, fpp, tmoney, wmoney, balance, monthly_vpp, yearly_vpp, total_fpp, total_tmoney, total_wmoney, ident, name, speed, format, limit, type, buyin, rake):
        'Create new instance of _Tournament(date, action, players, game, currency, amount, vpp, fpp, tmo